# Introduction

From last notebook, where we get the raw data from 2020-01-01 to 2021-03-31, we cleaned up the dataset by keeping the rows that have "bbl", right time frame (2020-03-15 -- 2021-03-21), only keeping the relevant complaint types, and many other tiny basic cleaning procedures.

In this notebook, our goal is to see, during this period, if any complaint types have strong correlations with evictions

### Hypothesis 1:

The buildings that have more 'COVID-19 Non-essential Construction' complaints had more evictions during 2020-03-15 -- 2021-03-21.

### Hypothesis 2:

The buildings that have more noise (any-type) complaints had more evictions during 2020-03-15 -- 2021-03-21.

### Hypothesis 3:

The buildings that have more water leak complaints had more evictions during 2020-03-15 -- 2021-03-21.

### Hypothesis 4:

The buildings that have more Appliance complaints had more evictions during 2020-03-15 -- 2021-03-21.


In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import io
import geopandas as gpd
import seaborn as sns

# suppress warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

#### As covid has passed for a while, this short study is not to examine what covid does to the relationships between landlord behavior and 311 complaints. Rather, only during covid, 311 complaints collected data on noise related to non-essentail constructions. Therefore, that period of time, March 2021 to March 2022 was a good time windown to look into how the non-essential construction noise might relate to unfriendly landloard behaviors.

#### The date of the data here is only from 2020 March to 2021 March.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# introducing filtered_df_covid data
file_path = '/content/drive/My Drive/X999/filtered_df_2023_2024.csv'

In [ ]:
covid_geo_df_raw = pd.read_csv(file_path)

In [ ]:
# make a copy
covid_geo_df = covid_geo_df_raw.copy()

In [ ]:
# already done in previous notebook
# covid_geo_df.created_date = pd.to_datetime(covid_geo_df.created_date)

In [ ]:
# covid_geo_df.closed_date = pd.to_datetime(covid_geo_df.closed_date)

In [ ]:
covid_geo_df.created_date.min(), covid_geo_df.created_date.max(), \
covid_geo_df.closed_date.min(), covid_geo_df.closed_date.max()

('2023-01-01 14:49:28',
 '2024-09-20 01:33:18',
 '2023-01-01 15:18:12',
 '2024-09-20 02:22:23')

In [ ]:
covid_geo_df.shape, \
covid_geo_df.info(), \
covid_geo_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2473148 entries, 0 to 2473147
Data columns (total 24 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   unique_key                      int64  
 1   created_date                    object 
 2   closed_date                     object 
 3   complaint_type                  object 
 4   descriptor                      object 
 5   location_type                   object 
 6   incident_zip                    float64
 7   incident_address                object 
 8   street_name                     object 
 9   cross_street_1                  object 
 10  cross_street_2                  object 
 11  intersection_street_1           object 
 12  intersection_street_2           object 
 13  address_type                    object 
 14  city                            object 
 15  status                          object 
 16  resolution_description          object 
 17  resolution_action_updated_d

((2473148, 24),
 None,
          unique_key  incident_zip           bbl      latitude     longitude  \
 count  2.473148e+06  2.473101e+06  2.473148e+06  2.473103e+06  2.473103e+06   
 mean   5.941704e+07  1.072384e+04  2.481924e+09  4.075464e+01 -7.392141e+01   
 std    1.730852e+06  5.324620e+02  1.109163e+09  8.639997e-02  6.274427e-02   
 min    5.641249e+07  1.000000e+04  0.000000e+00  4.049921e+01 -7.425322e+01   
 25%    5.790985e+07  1.030400e+04  2.023640e+09  4.068085e+01 -7.395613e+01   
 50%    5.946708e+07  1.046800e+04  2.048330e+09  4.075561e+01 -7.392455e+01   
 75%    6.084784e+07  1.122500e+04  3.055150e+09  4.083363e+01 -7.388767e+01   
 max    6.250461e+07  1.234500e+04  5.270001e+09  4.091287e+01 -7.370074e+01   
 
           zip_codes  
 count  2.467419e+06  
 mean   1.403006e+04  
 std    3.330820e+03  
 min    1.009000e+04  
 25%    1.160500e+04  
 50%    1.309200e+04  
 75%    1.686100e+04  
 max    2.600100e+04  )

In [ ]:
# since we particularly have an interest in "Non-Essentail Covid Consturction noise" that was only
# present during covid, we particularly set the timeframe as 2020-03-15 to 2021-03-31
# already didd this step in previous notebook
# filtered_df = covid_geo_df[covid_geo_df['created_date'] > pd.Timestamp('2020-03-15 00:00:00')]

In [ ]:
filtered_df = covid_geo_df.copy()
filtered_df.shape, \
filtered_df.info(), \
filtered_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2473148 entries, 0 to 2473147
Data columns (total 24 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   unique_key                      int64  
 1   created_date                    object 
 2   closed_date                     object 
 3   complaint_type                  object 
 4   descriptor                      object 
 5   location_type                   object 
 6   incident_zip                    float64
 7   incident_address                object 
 8   street_name                     object 
 9   cross_street_1                  object 
 10  cross_street_2                  object 
 11  intersection_street_1           object 
 12  intersection_street_2           object 
 13  address_type                    object 
 14  city                            object 
 15  status                          object 
 16  resolution_description          object 
 17  resolution_action_updated_d

((2473148, 24),
 None,
          unique_key  incident_zip           bbl      latitude     longitude  \
 count  2.473148e+06  2.473101e+06  2.473148e+06  2.473103e+06  2.473103e+06   
 mean   5.941704e+07  1.072384e+04  2.481924e+09  4.075464e+01 -7.392141e+01   
 std    1.730852e+06  5.324620e+02  1.109163e+09  8.639997e-02  6.274427e-02   
 min    5.641249e+07  1.000000e+04  0.000000e+00  4.049921e+01 -7.425322e+01   
 25%    5.790985e+07  1.030400e+04  2.023640e+09  4.068085e+01 -7.395613e+01   
 50%    5.946708e+07  1.046800e+04  2.048330e+09  4.075561e+01 -7.392455e+01   
 75%    6.084784e+07  1.122500e+04  3.055150e+09  4.083363e+01 -7.388767e+01   
 max    6.250461e+07  1.234500e+04  5.270001e+09  4.091287e+01 -7.370074e+01   
 
           zip_codes  
 count  2.467419e+06  
 mean   1.403006e+04  
 std    3.330820e+03  
 min    1.009000e+04  
 25%    1.160500e+04  
 50%    1.309200e+04  
 75%    1.686100e+04  
 max    2.600100e+04  )

In [ ]:
filtered_df.complaint_type.unique()

array(['WATER LEAK', 'HEAT/HOT WATER', 'UNSANITARY CONDITION',
       'Noise - Residential', 'Noise - Street/Sidewalk', 'Rodent',
       'Noise - Vehicle', 'Noise - Helicopter', 'Elevator',
       'Noise - Commercial', 'Smoking', 'General Construction/Plumbing',
       'DOOR/WINDOW', 'ELEVATOR', 'PAINT/PLASTER', 'Encampment',
       'PLUMBING', 'Animal-Abuse', 'ELECTRIC', 'GENERAL', 'Noise',
       'Non-Emergency Police Matter', 'FLOORING/STAIRS',
       'Indoor Air Quality', 'Illegal Fireworks', 'Graffiti', 'SAFETY',
       'APPLIANCE', 'Noise - Park', 'Homeless Person Assistance',
       'Urinating in Public', 'Non-Residential Heat', 'Mold',
       'Unsanitary Animal Pvt Property', 'Drinking',
       'Unsanitary Pigeon Condition', 'Asbestos', 'Plumbing',
       'Drinking Water', 'OUTSIDE BUILDING', 'Noise - House of Worship',
       'Indoor Sewage', 'Panhandling', 'Disorderly Youth',
       'Posting Advertisement', 'Bike/Roller/Skate Chronic',
       'Construction Safety Enforcement'

In [ ]:
# introduce evictions data
file_path = '/content/drive/My Drive/X999/evictions_df_cleaned.csv'

In [ ]:
evictions_df_raw = pd.read_csv(file_path)

In [ ]:
evictions_df_raw.executed_date = pd.to_datetime(evictions_df_raw.executed_date)

In [ ]:
evictions_df_raw.executed_date.min(), evictions_df_raw.executed_date.max()

(Timestamp('2017-01-03 00:00:00'), Timestamp('2024-12-30 00:00:00'))

In [ ]:
# # attempted to filter out only the covid times
# filtered_evictions_df = evictions_df_raw[
#     (evictions_df_raw['executed_date'] >= '2020-03-15') &
#     (evictions_df_raw['executed_date'] <= '2021-03-31')
# ]
# attempted to filter out only the covid times
filtered_evictions_df = evictions_df_raw[
    (evictions_df_raw['executed_date'] >= '2023-01-01') &
    (evictions_df_raw['executed_date'] <= '2024-09-20')
]

In [ ]:
# because the data is sooo few, I would still use all the eictions data from 2017-01-03 and 2024-09-16 to
# see if there is any correlations between evictions and complaint types.
filtered_evictions_df.head()

,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,eviction_postcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,bbl,nta
56287,317780/22,14733,420 KENT AVENUE,1822,2023-01-03,BROOKLYN,11249,Not an Ejectment,Possession,40.709915,-73.968525,1.0,33.0,547.0,3425676.0,3.021340e+09,North Side-South Side
56288,309216/21,108292,1410 NEW YORK AVENUE,6D,2023-01-03,BROOKLYN,11210,Not an Ejectment,Possession,40.638254,-73.945326,17.0,45.0,788.0,3327406.0,3.049950e+09,Flatbush
56289,056288/20,209909,98-30 57TH AVENUE,6A,2023-01-03,QUEENS,11368,Not an Ejectment,Possession,40.738027,-73.861173,4.0,21.0,455.0,4047321.0,4.019180e+09,Corona
56290,323414/22,109950,1418 BROOKLYN AVENUE,6E,2023-01-03,BROOKLYN,11210,Not an Ejectment,Possession,40.638315,-73.942444,17.0,45.0,782.0,3327343.0,3.049810e+09,East Flatbush-Farragut
56291,323504/22,109936,350 STERLING STREET,6H,2023-01-03,BROOKLYN,11225,Not an Ejectment,Possession,40.663206,-73.949469,9.0,40.0,329.0,3035095.0,3.013210e+09,Prospect Lefferts Gardens-Wingate


In [ ]:
# good to know, but would still use all the data.
filtered_evictions_df.shape

(20259, 17)

In [ ]:
# make a copy
evictions_df = evictions_df_raw.copy()

In [ ]:
# much more data and satisfy the gaussian distribution rule
evictions_df.shape

(80420, 17)

In [ ]:
evictions_df.shape, \
evictions_df.info(), \
evictions_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80420 entries, 0 to 80419
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   court_index_number         80420 non-null  object        
 1   docket_number              80420 non-null  int64         
 2   eviction_address           80420 non-null  object        
 3   eviction_apartment_number  80420 non-null  object        
 4   executed_date              80420 non-null  datetime64[ns]
 5   borough                    80420 non-null  object        
 6   eviction_postcode          80420 non-null  int64         
 7   ejectment                  80420 non-null  object        
 8   eviction/legal_possession  80420 non-null  object        
 9   latitude                   80420 non-null  float64       
 10  longitude                  80420 non-null  float64       
 11  community_board            80420 non-null  float64       
 12  coun

((80420, 17),
 None,
        docket_number                  executed_date  eviction_postcode  \
 count   80420.000000                          80420        80420.00000   
 mean   140498.980913  2020-05-09 18:09:40.154190336        10774.79086   
 min         1.000000            2017-01-03 00:00:00        10000.00000   
 25%     34837.250000            2018-02-09 00:00:00        10454.00000   
 50%     96608.500000            2019-04-18 00:00:00        10470.00000   
 75%    157263.750000            2023-06-20 00:00:00        11226.00000   
 max    496987.000000            2024-12-30 00:00:00        11698.00000   
 std    137631.116383                            NaN          506.47914   
 
            latitude     longitude  community_board  council_district  \
 count  80420.000000  80420.000000     80420.000000      80420.000000   
 mean      40.752220    -73.910144         8.012435         24.004601   
 min       40.499050    -74.251262         1.000000          1.000000   
 25%      

In [ ]:
evictions_df.columns

Index(['court_index_number', 'docket_number', 'eviction_address',
       'eviction_apartment_number', 'executed_date', 'borough',
       'eviction_postcode', 'ejectment', 'eviction/legal_possession',
       'latitude', 'longitude', 'community_board', 'council_district',
       'census_tract', 'bin', 'bbl', 'nta'],
      dtype='object')

In [ ]:
filtered_df.columns

Index(['unique_key', 'created_date', 'closed_date', 'complaint_type',
       'descriptor', 'location_type', 'incident_zip', 'incident_address',
       'street_name', 'cross_street_1', 'cross_street_2',
       'intersection_street_1', 'intersection_street_2', 'address_type',
       'city', 'status', 'resolution_description',
       'resolution_action_updated_date', 'bbl', 'borough', 'latitude',
       'longitude', 'location', 'zip_codes'],
      dtype='object')

In [ ]:
# just take one example to check (rather randomly)
truck_noise = filtered_df[filtered_df['complaint_type'] == 'Noise - Street/Sidewalk']
truck_noise.head()

,unique_key,created_date,closed_date,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,...,city,status,resolution_description,resolution_action_updated_date,bbl,borough,latitude,longitude,location,zip_codes
4,56416856,2023-01-01 14:55:05,2023-01-01 15:52:47,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11207.0,609 SUTTER AVENUE,SUTTER AVENUE,GEORGIA AVENUE,...,BROOKLYN,Closed,The Police Department reviewed your complaint ...,01/01/2023 03:52:53 PM,3.037530e+09,BROOKLYN,40.669197,-73.896452,"(40.66919734323511, -73.8964524840768)",17214.0
10,56419277,2023-01-01 14:57:09,2023-01-02 03:19:48,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10460.0,1815 WATERLOO PLACE,WATERLOO PLACE,CROTONA PARK NORTH,...,BRONX,Closed,The Police Department responded and upon arriv...,01/02/2023 03:19:54 AM,2.029580e+09,BRONX,40.840223,-73.887784,"(40.840222637101164, -73.8877836066035)",11269.0
17,56414917,2023-01-01 15:01:36,2023-01-01 15:27:47,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11412.0,111-25 202 STREET,202 STREET,111 AVENUE,...,SAINT ALBANS,Closed,The Police Department responded to the complai...,01/01/2023 03:27:57 PM,4.109601e+09,QUEENS,40.704101,-73.754504,"(40.7041014888078, -73.75450420635116)",15312.0
20,56415864,2023-01-01 15:02:40,2023-01-01 21:05:47,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11417.0,105-45 CROSSBAY BOULEVARD,CROSSBAY BOULEVARD,94 STREET,...,OZONE PARK,Closed,The Police Department responded and upon arriv...,01/01/2023 09:05:50 PM,4.091630e+09,QUEENS,40.679294,-73.844343,"(40.6792943237459, -73.84434330274757)",24013.0
24,56414962,2023-01-01 15:05:27,2023-01-01 17:45:24,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10472.0,1202 NOBLE AVENUE,NOBLE AVENUE,GLEASON AVENUE,...,BRONX,Closed,The Police Department responded to the complai...,01/01/2023 05:45:27 PM,2.037600e+09,BRONX,40.829689,-73.869730,"(40.82968865996156, -73.86973033481145)",11610.0


In [ ]:
filtered_df.complaint_type.unique()

array(['WATER LEAK', 'HEAT/HOT WATER', 'UNSANITARY CONDITION',
       'Noise - Residential', 'Noise - Street/Sidewalk', 'Rodent',
       'Noise - Vehicle', 'Noise - Helicopter', 'Elevator',
       'Noise - Commercial', 'Smoking', 'General Construction/Plumbing',
       'DOOR/WINDOW', 'ELEVATOR', 'PAINT/PLASTER', 'Encampment',
       'PLUMBING', 'Animal-Abuse', 'ELECTRIC', 'GENERAL', 'Noise',
       'Non-Emergency Police Matter', 'FLOORING/STAIRS',
       'Indoor Air Quality', 'Illegal Fireworks', 'Graffiti', 'SAFETY',
       'APPLIANCE', 'Noise - Park', 'Homeless Person Assistance',
       'Urinating in Public', 'Non-Residential Heat', 'Mold',
       'Unsanitary Animal Pvt Property', 'Drinking',
       'Unsanitary Pigeon Condition', 'Asbestos', 'Plumbing',
       'Drinking Water', 'OUTSIDE BUILDING', 'Noise - House of Worship',
       'Indoor Sewage', 'Panhandling', 'Disorderly Youth',
       'Posting Advertisement', 'Bike/Roller/Skate Chronic',
       'Construction Safety Enforcement'

In [ ]:
# covid_construction_complaints_df = filtered_df[filtered_df['complaint_type'] == 'COVID-19 Non-essential Construction']
# covid_construction_complaints_df.head()

In [ ]:
# covid_complaint_count = covid_construction_complaints_df.groupby('bbl').size().reset_index(name='covid_complaint_count')


In [ ]:
# covid_complaint_count.head()

In [ ]:
eviction_count = evictions_df.groupby('bbl').size().reset_index(name='eviction_count')
eviction_count.head()

,bbl,eviction_count
0,0.000000e+00,3
1,1.000078e+09,2
2,1.000158e+09,4
3,1.000158e+09,3
4,1.000160e+09,2


In [ ]:
# merged_df = covid_complaint_count.merge(eviction_count, on='bbl', how='inner')
# merged_df.head()
merged_df = filtered_df.merge(filtered_evictions_df, on='bbl', how='inner')
merged_df.head()

,unique_key,created_date,closed_date,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,...,eviction_postcode,ejectment,eviction/legal_possession,latitude_y,longitude_y,community_board,council_district,census_tract,bin,nta
0,56413440,2023-01-01 14:49:28,2023-01-05 09:33:33,WATER LEAK,SLOW LEAK,RESIDENTIAL BUILDING,10462.0,2075 ST RAYMONDS AVENUE,ST RAYMONDS AVENUE,NaN,...,10462,Not an Ejectment,Possession,40.837554,-73.854677,9.0,18.0,21002.0,2096728.0,Parkchester
1,56413440,2023-01-01 14:49:28,2023-01-05 09:33:33,WATER LEAK,SLOW LEAK,RESIDENTIAL BUILDING,10462.0,2075 ST RAYMONDS AVENUE,ST RAYMONDS AVENUE,NaN,...,10462,Not an Ejectment,Possession,40.838248,-73.859128,9.0,18.0,21002.0,2096696.0,Parkchester
2,56413440,2023-01-01 14:49:28,2023-01-05 09:33:33,WATER LEAK,SLOW LEAK,RESIDENTIAL BUILDING,10462.0,2075 ST RAYMONDS AVENUE,ST RAYMONDS AVENUE,NaN,...,10462,Not an Ejectment,Possession,40.839754,-73.857904,9.0,18.0,21002.0,2096748.0,Parkchester
3,56413440,2023-01-01 14:49:28,2023-01-05 09:33:33,WATER LEAK,SLOW LEAK,RESIDENTIAL BUILDING,10462.0,2075 ST RAYMONDS AVENUE,ST RAYMONDS AVENUE,NaN,...,10462,Not an Ejectment,Possession,40.840033,-73.857238,9.0,18.0,21002.0,2096742.0,Parkchester
4,56413440,2023-01-01 14:49:28,2023-01-05 09:33:33,WATER LEAK,SLOW LEAK,RESIDENTIAL BUILDING,10462.0,2075 ST RAYMONDS AVENUE,ST RAYMONDS AVENUE,NaN,...,10462,Not an Ejectment,Possession,40.840464,-73.855441,9.0,18.0,21002.0,2096739.0,Parkchester


In [ ]:
# # not high correlation
# correlation = merged_df['covid_complaint_count'].corr(merged_df['eviction_count'])
# print(f"Correlation between COVID-19 Non-essential Construction complaints and evictions: {correlation}")

In [ ]:
# plt.scatter(merged_df['covid_complaint_count'], merged_df['eviction_count'])
# plt.xlabel('Number of COVID-19 Non-essential Construction Complaints')
# plt.ylabel('Number of Evictions')
# plt.title('Scatter Plot of Complaints vs Evictions')
# plt.show()

# # no strong correlation

In [ ]:
# checkout the relationship between other complaint types and the evictions data:
covid_complaints_count = filtered_df.groupby(['bbl', 'complaint_type']).size().reset_index(name='complaint_count')

In [ ]:
eviction_count = filtered_evictions_df.groupby('bbl').size().reset_index(name='eviction_count')

In [ ]:
merged_df = covid_complaints_count.merge(eviction_count, on='bbl', how='inner')

In [ ]:
# using dict directly, better for sorting the values
dict_of_correlations = {}

unique_complaints = merged_df['complaint_type'].unique()

for complaint in unique_complaints:
    # each complaint type (each type stands alone as a df)
    complaint_df = merged_df[merged_df['complaint_type'] == complaint]
    if len(complaint_df) > 1:
        correlation = complaint_df['complaint_count'].corr(complaint_df['eviction_count'])
        dict_of_correlations[complaint] = correlation

In [ ]:
# sort
sorted_correlations = dict(sorted(dict_of_correlations.items(), key=lambda item: item[1], reverse=False))

for complaint_type, correlation in sorted_correlations.items():
    print(f"complaint type: {complaint_type}, correlation with evictions: {correlation}")

complaint type: Disorderly Youth, correlation with evictions: -0.16888013236829955
complaint type: Bike/Roller/Skate Chronic, correlation with evictions: -0.1006587427289815
complaint type: Noise - House of Worship, correlation with evictions: -0.0948791737588852
complaint type: Mosquitoes, correlation with evictions: -0.07962575371800426
complaint type: Noise - Commercial, correlation with evictions: -0.028933648577882623
complaint type: Encampment, correlation with evictions: -0.0242302692183192
complaint type: Noise - Park, correlation with evictions: -0.022640268775617232
complaint type: Homeless Person Assistance, correlation with evictions: -0.01414119326912577
complaint type: Flooring/Stairs, correlation with evictions: -0.011558932152722436
complaint type: Electric, correlation with evictions: -0.007681464149145969
complaint type: Drinking, correlation with evictions: 0.008784319133171174
complaint type: Rodent, correlation with evictions: 0.010798291790075212
complaint type: O

Top 5 2023-2024 \
complaint type: Elevator, correlation with evictions: 0.3841365951351973 \
complaint type: PLUMBING, correlation with evictions: 0.4053484001615624 \
complaint type: PAINT/PLASTER, correlation with evictions: 0.42973583835471096 \
complaint type: APPLIANCE, correlation with evictions: 0.43161398824366837 \
complaint type: Drinking Water, correlation with evictions: 0.49347596167451807 \

If we look at some combinations of these attributes, we may be able to see some stronger correlations between the complaints and evictions.

## conclusion:
The top 5 most correlated:  

complaint type: Day Care, correlation with evictions: 0.984057367453252 \
complaint type: Posting Advertisement, correlation with evictions: 0.9524850821144154 \
complaint type: Panhandling, correlation with evictions: 0.9098431565585486 \
complaint type: WATER LEAK, correlation with evictions: 0.45215453304219416 \
complaint type: APPLIANCE, correlation with evictions: 0.38966445546749745

## Results:

### Hypothesis 1: (False)

The buildings that have more 'COVID-19 Non-essential Construction' complaints had more evictions during 2020-03-15 -- 2021-03-21.

### Hypothesis 2: (False)

The buildings that have more noise (any-type) complaints had more evictions during 2020-03-15 -- 2021-03-21.

### Hypothesis 3: (True)

The buildings that have more water leak complaints had more evictions during 2020-03-15 -- 2021-03-21.

### Hypothesis 4: (True)

The buildings that have more Appliance complaints had more evictions during 2020-03-15 -- 2021-03-21.

